https://www.edduby.com/tutorial/how-to-make-menu-in-python/?utm_source=chatgpt.com

In [ ]:
# Import the MySQL connector module and error handling class
import mysql.connector as mydbconnection
from mysql.connector import Error

# --------------------------------------------------
# Function: connect_to_db
# Purpose: Establish and return a connection to the MySQL database
# --------------------------------------------------
def connect_to_db():
    return mydbconnection.connect(
        host="localhost",        # Database host (usually localhost)
        port="3306",             # Default MySQL port
        user="root",             # MySQL username
        password="password",     # MySQL password
        database="creditcard_capstone"  # Name of the database to connect to
    )

# --------------------------------------------------
# Function: main
# Purpose: Display main menu and handle user input
# --------------------------------------------------
def main():
    while True:  # Loop continuously until the user chooses to quit
        print(
            "\n--- Main Menu ---"
            "\n1. Transactions"     # Option 1: View transaction details
            "\n2. Customer"         # Option 2: View customer details
            "\n3. Quit"             # Option 3: Exit the program
        )

        # Prompt user for their choice
        choice = input("Please Enter Your Selection (1, 2, or 3): ")

        # Use match-case to handle the selected menu option
        match choice:
            case "1":
                print("\n--- Transactions Details ---")
                transaction()  # Call the function to handle transaction details
            case "2":
                print("\n--- Customer Details ---")
                customer() # Call the function to handle customer details
            case "3":
                print("Goodbye!")  # Exit message
                break             # Exit the loop and end the program
            case _:
                print("Invalid choice. Please try again.")  # Catch all other inputs

# --------------------------------------------------
# Function: transaction
# Purpose: Prompt for ZIP/month/year and retrieve matching transactions
# --------------------------------------------------
def transaction():
    # ---------------------------
    # STEP 1: Prompt user for ZIP code
    # ---------------------------
    while True:
        zip_code = input("Please enter your 5-digit ZIP Code (e.g., 12345): ")
        
        # Check if the input is exactly 5 digits and numeric
        if len(zip_code) == 5 and zip_code.isdigit():
            break  # Valid ZIP code — exit loop
        else:
            print("Invalid ZIP code. Try again.")  # Ask again on invalid input

    # ---------------------------
    # STEP 2: Prompt user for month
    # ---------------------------
    while True:
        month_input = input("Please enter the month (MM format, e.g., 03): ")
        
        # Check if input is a 2-digit number between 01 and 12
        if len(month_input) == 2 and month_input.isdigit() and 1 <= int(month_input) <= 12:
            break  # Valid month — exit loop
        else:
            print("Invalid month. Enter a value between 01 and 12.")  # Invalid input

    # ---------------------------
    # STEP 3: Prompt user for year
    # ---------------------------
    while True:
        year_input = input("Please enter the year (YYYY format, e.g., 2022): ")
        
        # Check if input is a 4-digit number
        if len(year_input) == 4 and year_input.isdigit():
            break  # Valid year — exit loop
        else:
            print("Invalid year. Enter a valid 4-digit year.")  # Invalid input

    # ---------------------------
    # STEP 4: Connect to DB and run query
    # ---------------------------
    try:
        # Establish connection to the database
        conn = connect_to_db()

        # Create a cursor to execute SQL queries
        # dictionary=True makes each row a dictionary (e.g., row['TRANSACTION_VALUE'])
        cursor = conn.cursor(dictionary=True)

        # SQL query with parameter placeholders (%s)
        # Joins credit card transactions with customer data based on SSN
        query = """
            SELECT
                cc.TRANSACTION_ID,
                cc.CREDIT_CARD_NO,
                cc.TRANSACTION_TYPE,
                cc.TRANSACTION_VALUE,
                cc.DAY,
                cc.MONTH,
                cc.YEAR
            FROM CDW_SAPP_CREDIT_CARD cc
            JOIN CDW_SAPP_CUSTOMER c ON cc.CUST_SSN = c.SSN
            WHERE c.CUST_ZIP = %s AND cc.MONTH = %s AND cc.YEAR = %s
            ORDER BY cc.DAY DESC;
        """

        # Execute query with actual user inputs (safely substitutes into %s)
        cursor.execute(query, (zip_code, int(month_input), int(year_input)))

        # Fetch all results into a list of dictionaries
        results = cursor.fetchall()

        # ---------------------------
        # STEP 5: Display results
        # ---------------------------
        if results:
            print(f"\nTransactions for ZIP {zip_code} in {month_input}/{year_input}:\n")

            # Loop through each row in the result and print formatted data
            for row in results:
                print(
                    f"Transaction ID: {row['TRANSACTION_ID']}, "
                    f"Card No: {row['CREDIT_CARD_NO']}, "
                    f"Type: {row['TRANSACTION_TYPE']}, "
                    f"Amount: ${row['TRANSACTION_VALUE']:.2f}, "
                    f"Date: {row['MONTH']:02d}/{row['DAY']:02d}/{row['YEAR']}"
                )
        else:
            # No results found for the input ZIP/month/year
            print("No transactions found for that ZIP code and date.")

    # ---------------------------
    # STEP 6: Error handling
    # ---------------------------
    except Error as e:
        # Catch and print database connection or execution errors
        print(f"Database error: {e}")

    # ---------------------------
    # STEP 7: Always close connection (even on error)
    # ---------------------------
    finally:
        if conn.is_connected():
            cursor.close()  # Close the cursor
            conn.close()    # Close the database connection

# --------------------------------------------------
# Entry Point
# Ensures that main() runs only when this script is executed directly
# --------------------------------------------------
if __name__ == "__main__":
    main()



--- Main Menu ---
1. Transactions
2. Customer
3. Quit

--- Transactions Details ---

Transactions for ZIP 17201 in 03/2018:

Transaction ID: 29034, Card No: 4210653358330252, Type: Entertainment, Amount: $56.25, Date: 03/28/2018
Transaction ID: 14145, Card No: 4210653373620931, Type: Test, Amount: $29.74, Date: 03/27/2018
Transaction ID: 14154, Card No: 4210653373620931, Type: Entertainment, Amount: $59.41, Date: 03/27/2018
Transaction ID: 14139, Card No: 4210653373620931, Type: Entertainment, Amount: $56.38, Date: 03/26/2018
Transaction ID: 29031, Card No: 4210653358330252, Type: Grocery, Amount: $27.65, Date: 03/26/2018
Transaction ID: 35377, Card No: 4210653389909043, Type: Gas, Amount: $54.78, Date: 03/25/2018
Transaction ID: 44796, Card No: 4210653320012911, Type: Grocery, Amount: $11.11, Date: 03/24/2018
Transaction ID: 29075, Card No: 4210653358330252, Type: Grocery, Amount: $16.09, Date: 03/24/2018
Transaction ID: 29033, Card No: 4210653358330252, Type: Bills, Amount: $88.12, 

KeyboardInterrupt: Interrupted by user